In [1]:
import os
import torch
from tqdm import tqdm
from vllm import LLM, SamplingParams
from huggingface_hub import login
import json
import csv
from tqdm import tqdm
with open('/burg/home/nd2794/keys.json', 'r') as file:
    keys = json.load(file)

/burg/dsi/users/nd2794/conda_envs/tr-llama3-vllm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-30 12:04:55,610	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
cache_dir = '/burg/dsi/users/nd2794/HF_HOME'
os.environ['HF_HOME'] = cache_dir
hf_token = keys["hf_read_token"]

In [3]:
login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /burg/home/nd2794/.cache/huggingface/token
Login successful


In [4]:
model_id = 'llama3-8b-inst'
model_name = 'meta-llama/Meta-Llama-3-8B-Instruct'

In [5]:
temperature = 0.7
sampling_params = SamplingParams(top_p=0.9, temperature=temperature, max_tokens=3000)

In [6]:
llm = LLM(
    model=model_name, 
    tensor_parallel_size=2, 
    download_dir=cache_dir
)

INFO 07-30 12:04:57 config.py:715] Defaulting to use mp for distributed inference
INFO 07-30 12:04:57 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir='/burg/dsi/users/nd2794/HF_HOME', load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct, use_v2_block_manager=False, enable_prefix_cach

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:08<00:25,  8.64s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:27<00:28, 14.41s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [01:02<00:24, 24.01s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [01:06<00:00, 15.91s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [01:06<00:00, 16.50s/it]


INFO 07-30 12:06:17 model_runner.py:692] Loading model weights took 7.4829 GB
(VllmWorkerProcess pid=856172) INFO 07-30 12:06:17 model_runner.py:692] Loading model weights took 7.4829 GB


INFO 07-30 12:06:28 distributed_gpu_executor.py:56] # GPU blocks: 26952, # CPU blocks: 4096
INFO 07-30 12:06:30 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-30 12:06:30 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=856172) INFO 07-30 12:06:30 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=856172) INFO 07-30 12:06:30 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory 

In [7]:
tokenizer = llm.get_tokenizer()

In [9]:
system_content = """You will be given two texts which describe two different events. 
The two events may be from the same context or they can be unrelated to each other. 
We call the two events related if they are from the same context and have a temporal relation (one event starts or ends before another. Event windows can be overlapping). 
We call the two events are unrealted if they are from very different contexts. 
Your task is to determine if the two events have a likely temporal relation, meaning one procedes the other.

Each problem has all of the following information:
- Description of two events, each in a different line."""

user_prompt_1 = """Event 1: The man in the grey and white shirt enters the enclosed squash court picks up some of the balls and proceeds to load the squash cannon serving machines .
Event 2: A man wearing a white and grey shirt serves in a practice squash session and another man wearing a purple shirt returns the serves in an enclosed squash court .

Do the two events summarized above have a temporal relationship? Please choose between "Yes" and "No" only. Please answer in JSON format with explanation."""

assistant_response_1 = """{"answer": "Yes", "explanation": "The serving machine must be loaded with balls before the man can practice serving."}"""

user_prompt_2 = """Event 1: Sumo wrestlers lift up legs and then crouch .
Event 2: Sumo wrestlers eat food in the dojo .

Do the two events summarized above have a temporal relationship? Please choose between "Yes" and "No" only. Please answer in JSON format with explanation."""

assistant_response_2 = """{"answer": "No", "explanation": "It is ambiguous which event comes first as neither must necessarily precede the other."}"""

INSTRUCTION = """Do the two events summarized above have a temporal relationship? Please choose between "Yes" and "No" only. Please answer in JSON format with explanation."""

In [10]:
def run_llama_subset():
    fname = '../data_viz/samples_200.csv'
    with open(fname, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        data = list(reader)

    llama_outputs = []

    for sample in tqdm(data[1:]):
        idx = sample[0]
        user_prompt_3 = 'Event 1: ' + sample[1] + '\nEvent 2: ' + sample[2] + '\n\n' + INSTRUCTION

        messages = [
            {"role": "system", "content": system_content},
            {"role": "user", "content": user_prompt_1},
            {"role": "assistant", "content": assistant_response_1},
            {"role": "user", "content": user_prompt_2},
            {"role": "assistant", "content": assistant_response_2},
            {"role": "user", "content": user_prompt_3}
        ]

        instruction = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        prompt_token_ids = tokenizer.encode(instruction, return_tensors="pt").tolist()
    
        ans = llm.generate(
            prompt_token_ids=prompt_token_ids, 
            sampling_params=sampling_params
        )

        answer_json_str1 = ans[0].outputs[0].text
        
        try:
            answer_json = json.loads(answer_json_str1)
            answer = answer_json['answer']
            explanation = answer_json['explanation']
        except:
            try:
                last_inst_idx = output.rfind('{')
                answer_json_str2 = output[last_inst_idx:]
                answer_json = json.loads(answer_json_str2)
                answer = answer_json['answer']
                explanation = answer_json['explanation']
            except:
                answer = 'unclear'
                explanation = 'none'

        sample[-1] = answer
        sample.append(explanation)
        
    with open('../data_viz/samples_200_llama3_prompt2_temp_{}.csv'.format(temperature), 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(data)
    return   

In [11]:
run_llama_subset()

100%|██████████| 200/200 [01:39<00:00,  2.01it/s]
